# Multithreading and Multiprocessing in Python

## The classic way

Let's import the libraries needed

In [ ]:
import multiprocessing
import threading
import time
from os import getpid, getppid

Then, define a simple task to be running concurrently

In [ ]:
# Simple example task
def task(n):
    # Refer to the global variable "sum"
    global sum
    # Print the process ID and the parent process ID
    print(f"Task {n} started ('Pid: {getpid()}, PPid: {getppid()})")    
    # Uncomment one of the two options:
    # 1. Sleep for 2 seconds
    time.sleep(2)
    # 2. Execute some CPU-intensive workload
    #for x in range(1, 20000000):
    #    float(x) / 3.141592  # Dividing x by Pi
    #    float(3.141592) / x  # Dividing the number Pi by x
    sum = sum + n
    print(f"Task {n} finished, sum is {sum}\n")


Let's run a simple task multiple times sequentially, i.e. without any concurrency

In [ ]:
sum = 0

def sequential_example():
    print("Sequential Example")
    for i in range(5):
        task(i)

sequential_example()

### The [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) library

Processes are independent execution environments with their own protected memory space and resources.
Multiple processes can leverage multiple cores for true parallel execution.

**Note:** this code snippet might not work inside the Notebook. You can run `multiprocess_example.py` from your terminal.

In [ ]:
sum = 0

# Function to demonstrate multiprocessing
def multiprocessing_example():
    print("Multiprocessing Example")
    processes = []
    for i in range(5):
        p = multiprocessing.Process(target=task, args=(i,))
        processes.append(p)
        p.start()
    
    # Wait for all processes to finish
    for p in processes:
        p.join()

multiprocessing_example()

### The [threading](https://docs.python.org/3/library/threading.html) library

Threads are lightweight units of execution within a single process.
They share the same memory space and resources of the process.
Multiple threads can run concurrently within the same process.

In [ ]:
sum = 0

# Function to demonstrate multithreading
def multithreading_example():
    print("Multithreading Example")
    threads = []
    for i in range(5):
        t = threading.Thread(target=task, args=(i,))
        threads.append(t)
        t.start()
    
    # Wait for all threads to finish
    for t in threads:
        t.join()

multithreading_example()

### Multiprocessing vs. Multithreading in Python

**CPython** is the reference implementation of the Python interpreter and compiler, written in C and Python. CPython's memory management is not thread-safe, i.e. it does not guarantee that the execution of a thread will not interfere with the concurrent execution of another thread on the shared memory space.

**Race Condition**: the condition of a system where the system's behavior is dependent on the sequence or timing of other, uncontrollable events, such as concurrent thread scheduling. 

The **Global Interpreter Lock** (GIL), is a mutual exclusion feature that prevents multiple threads from executing Python bytecodes at once. The GIL prevents race conditions and ensures thread safety.
Python's GIL limits true parallel execution on multi-core processors for CPU-bound tasks.
Only one thread can execute Python bytecode at a time within the GIL.

Multiprocessing is ideal for CPU-bound tasks that can be broken down into independent processes. Multithreading is still beneficial for tasks that involve waiting, e.g., sleeps tasks or I/O-bound tasks such as network requests and file R/W access.

## The [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) library

The `concurrent.futures` library provides a high-level interface for asynchronously executing concurrent tasks. It allows to create a pool of worker threads or processes.

The asynchronous execution can be performed with threads, using `ThreadPoolExecutor`, or separate processes, using `ProcessPoolExecutor`. Both implement the same interface, which is defined by the abstract `Executor` class.

In [ ]:
sum = 0

import concurrent.futures

# Multithreading with manual submit
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for i in range(5):
        executor.submit(task, i)


In [ ]:
sum = 0

# Multithreading with map function
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(task, list(range(5)))

In [ ]:
sum = 0

# Multiprocessing with map function
with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    executor.map(task, list(range(5)))